# Module 1: ROI Definition and Imagery Acquisition
Define the region of interest (ROI), setup data access to Copernicus, and download the relevant imagery.

1. SentinelHub must be installed: https://sentinelhub-py.readthedocs.io/en/latest/install.html
2. Access must be provided to the Copernicus Data Space Ecosystem (CDSE): https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Overview/Authentication.html

See Sentinel Hub Process API instructions for more details (https://sentinelhub-py.readthedocs.io/en/latest/examples/process_request_cdse.html#Credentials).v>

Package installation requirements:
- Matplotlib (`matplotlib`): https://pypi.org/project/matplotlib/
- NumPy (`numpy`): https://pypi.org/project/numpy/
- SentinelHub (`sentinelhub`): https://pypi.org/project/sentinelhub/

***
## User Input
Define the key inputs for the module here:
1. Region of interest (ROI) by defining minimum/maximum decimal latitude/longitude.
2. Start and end time in the format: yyyy-mm-dd
3. Data output location.
4. Data access credentials.

### Spatial/Temporal Extent

In [ ]:
# decimal latitude and longitude extents
# accra example
latmin = 5.4219
latmax = 5.5504
lonmin = -0.3547
lonmax = -0.1933

# start and end times in yyyy-mm-dd format
start_date = "2018-10-31" 
end_date = "2018-11-01"

### Data Output Location

In [ ]:
# define output location for the generated .tiff
output_folder = ''
output_name = 'test' #output file name

### Define CDSE Credentials
The **client ID** and **client secret** must be defined here to allow acces to the CDSE database. To do this, an OAuth client must be registered via the Sentinel Hub services dashboard (https://shapps.dataspace.copernicus.eu/dashboard/#/) to grant remote access to Copernicus. <br><br>For full instructions, visit: https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Overview/Authentication.html.
<br><br>
Provide the **client ID** and **client secret** as above. The **instance ID** is a string to describe the name of the authentication instance, which can be any relatively short string. The authentication profile can also be given a name to save for use later.

In [ ]:
client_id = ''
client_secret = ''
instance_id = 'sentinelhub_cid'
profile_name = 'test-profile'

***

## Load Packages and Define Functions

In [ ]:
from sentinelhub import WebFeatureService
from sentinelhub import (
    CRS, 
    BBox, 
    DataCollection, 
    SHConfig, 
    DownloadRequest,
    MimeType, 
    MosaickingOrder, 
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions
)
import matplotlib.pyplot as plt
import numpy as np
from typing import Any
import os

In [ ]:
# define image plotting function to display acquired imagery (from https://sentinelhub-py.readthedocs.io/en/latest/index.html)
def plot_image(
    image: np.ndarray, factor: float = 1.0, clip_range: tuple[float, float] | None = None, **kwargs: Any
) -> None:
    _, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 15))
    if clip_range is not None:
        ax.imshow(np.clip(image * factor, *clip_range), **kwargs)
    else:
        ax.imshow(image * factor, **kwargs)
    ax.set_xticks([])
    ax.set_yticks([])

## Construct Config File

In [ ]:
# define Copernicus Data Space Ecosystem (CDSE) profile
config = SHConfig()
config.sh_client_id = client_id
config.sh_client_secret = client_secret
config.sh_base_url = 'https://sh.dataspace.copernicus.eu'
config.sh_token_url = 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token'
config.instance_id = instance_id
config.save(profile_name)

SHConfig.get_config_location()

print(config)

## Setup ROI and Time Range
The ROI is defined using a bounding box, providing min/max extent for latitude and longitude as in the **User Input** section for Module 1.

In [ ]:
req_coords_wgs84 = (lonmin, latmin, lonmax, latmax)
resolution = 10
req_bbox = BBox(bbox=req_coords_wgs84, crs=CRS.WGS84)
req_size = bbox_to_dimensions(req_bbox, resolution=resolution)

print(f"Image shape at {resolution} m resolution: {req_size} pixels")

## Download and Display True Colour Image at ROI
The true colour image that covers the ROI is displayed here, for Sentinel-2 level 2A data.

In [ ]:
# image acquisition script (from https://sentinelhub-py.readthedocs.io/en/latest/examples/process_request_cdse.html#Credentials)
evalscript_true_color = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B02", "B03", "B04"]
            }],
            output: {
                bands: 3
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B04, sample.B03, sample.B02];
    }
"""

request_true_color = SentinelHubRequest(
    evalscript = evalscript_true_color,
    input_data = [
        SentinelHubRequest.input_data(
            DataCollection.SENTINEL2_L2A.define_from("s2l2a", service_url=config.sh_base_url),
            time_interval=(start_date, end_date),
        )
    ],
    responses = [SentinelHubRequest.output_response("default", MimeType.PNG)],
    bbox = req_bbox,
    size = req_size,
    config = config,
)

In [ ]:
%%time
# request true colour image
true_color_imgs = request_true_color.get_data()

In [ ]:
image = true_color_imgs[0]

# factor 1/255 to scale between 0-1
# factor 3.5 to increase brightness
plot_image(image, factor=3.5 / 255, clip_range=(0, 1))

## Download and Display False Colour Image at ROI
The false colour image that covers the ROI is displayed here, for Sentinel-2 level 2A data. The image is a mosaic of images collected over the specified date range.

In [ ]:
# image acquisition script (from https://sentinelhub-py.readthedocs.io/en/latest/examples/process_request_cdse.html#Credentials)
evalscript_all_bands = """
    //VERSION=3
    function setup() {
        return {
            input: [{
                bands: ["B01","B02","B03","B04","B05","B06","B07","B08","B8A","B09","B11","B12","SCL"],
                units: "DN"
            }],
            output: {
                bands: 13,
                sampleType: "INT16"
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B01,
                sample.B02,
                sample.B03,
                sample.B04,
                sample.B05,
                sample.B06,
                sample.B07,
                sample.B08,
                sample.B8A,
                sample.B09,
                sample.B11,
                sample.B12,
                sample.SCL];
    }
"""

request_all_bands = SentinelHubRequest(
    data_folder = output_folder,
    evalscript = evalscript_all_bands,
    input_data = [
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L2A.define_from("s2l2a", service_url=config.sh_base_url),
            time_interval=(start_date, end_date),
            mosaicking_order=MosaickingOrder.LEAST_CC,
        )
    ],
    responses = [SentinelHubRequest.output_response("default", MimeType.TIFF)],
    bbox = req_bbox,
    size = req_size,
    config = config,
)

In [ ]:
%%time
# request imagery with all bands
all_bands_response = request_all_bands.get_data()

In [ ]:
# plot false colour image using bands B03, B04, and B08
plot_image(all_bands_response[0][:, :, [2, 3, 7]], factor=3.5/1e4, clip_range=(0, 1))

## Save Dataset
Save the multispectral Sentinel-2 data as a GeoTIFF.

In [ ]:
%%time
request_all_bands.save_data()

for folder, _, filenames in os.walk(request_all_bands.data_folder):
    for filename in filenames:
        f = os.path.join(folder, filename)
        comp = f.split('\\')
        req_id = comp[-2] # request ID that is automatically generated

print("Generated request ID: "+req_id) # the request ID can be used to continue into Module 2

***
***
# Module 2: Read Data and Detect Objects
Read GeoTIFF data and perform object detection using K-Means clustering.

Package installation requirements:
- Matplotlib (`matplotlib`): https://pypi.org/project/matplotlib/
- NumPy (`numpy`): https://pypi.org/project/numpy/
- Rasterio (`rasterio`): https://pypi.org/project/rasterio/
- SciKit Learn (`scikit-learn`): https://pypi.org/project/scikit-learn/
- Xarray (`xarray`): https://pypi.org/project/xarray/0.8.0rc1/
- Rasterio Xarray Extension (`rioxarray`): https://pypi.org/project/rioxarray/

***
## User Input
Define the key inputs for the module here:
1. Path to the dataset.
2. K-means clustering details.

### Path to Data

In [ ]:
# define the path to the tiff output data from module 1 (or tiffs defined elsewhere)
try:
    req_id
except NameError:
    var_exists = False
    path_to_tiff = ''
else:
    var_exists = True

if var_exists:
    print('Found request ID generated from Module 1: '+req_id)
    path_to_tiff = f
else:
    if path_to_tiff == '':
        path_to_tiff = input('Please define the path to the GeoTIFF:')
    else:
        print('Path to GeoTIFF has been defined.')

### K-Means Setup
The number of clusters used for the K-means clustering are defined here, separately for clustering across the normalised difference vegetation index (NDVI) and the floating debris index (FDI).

In [ ]:
# define the number of clusters for the K-means algorithm
n_clusters_ndvi = 3
n_clusters_fdi = 3

# Output Location

In [ ]:
output_folder = ''

***
## Load Packages and Define Functions

In [ ]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np
from rasterio.plot import show, show_hist
import xarray as xr
import rioxarray
from sklearn.cluster import KMeans

## Load Data

In [ ]:
# open tiff with rasterio
data = rasterio.open(path_to_tiff)
xda = rioxarray.open_rasterio(path_to_tiff)
print('GeoTIFF opened, '+str(len(xda.band))+' bands found.')
xda

In [ ]:
# key dataset details
print(xda.rio.crs) # coordinate reference system
print(xda.rio.nodata) # nodata values
print(xda.rio.bounds()) # lat/lon bounds
print(xda.rio.width) # width of the image
print(xda.rio.height) # height of the image

In [ ]:
# define the relevant Sentinel-2 bands
BLU = xda.data[1] # blue band (B02)
GRN = xda.data[2] # green band (B03)
RED = xda.data[3] # red band (B04)
RE2 = xda.data[5] # second red edge band (B06)
NIR = xda.data[7] # near infrared (NIR) band (B08)
SWIR1 = xda.data[11] # first short wave infrared (SWIR) band (B11)
SWIR2 = xda.data[12] # second short wave infrared (SWIR) band (B12)

# define the spatial extent
lon = xda.x.data
lat = xda.y.data
extent = [np.min(lon), np.max(lon), np.max(lat), np.min(lat)]

# plot example RGB data
fig, ax = plt.subplots(1,3, figsize=(21,7))
cols = ('Blues', 'Greens', 'Reds') # colour maps for plotting
for i in np.arange(1,4):
    plt.subplot(1,3,i) 
    plt.imshow(xda.data[i], cmap=cols[i-1], extent=extent) # plot the appropriate band
    plt.title(cols[i-1][:-1]+' channel')

## K-Means Clustering (NDVI)
Clustering across the normalised difference vegetation index (NDVI).

In [ ]:
ndvi = (NIR-RED)/(NIR+RED) # normalised difference vegetation index (NDVI)
ndvi_clust = ndvi.reshape((-1, 1)) # reshape to allow clustering

# check for nodata
nd = np.isnan(ndvi)
ndvi[nd] = np.nanmean(ndvi) # rough mean filter to allow for K-means clustering

# perform K-means clustering
kmeans_ndvi = KMeans(n_clusters=n_clusters_ndvi)
kmeans_ndvi.fit(ndvi_clust)
centroids_ndvi = kmeans_ndvi.cluster_centers_
labels_ndvi = kmeans_ndvi.labels_

# reshape output data for plotting
debris_ndvi = np.choose(labels_ndvi, centroids_ndvi)
debris_ndvi.shape = ndvi.shape
labels_ndvi.shape = ndvi.shape

# plot results
fig, ax = plt.subplots(figsize=(8,8))
ax.set_title('NDVI Classification')
pos = ax.imshow(labels_ndvi, cmap=plt.get_cmap('viridis', n_clusters_ndvi), extent=extent)
cbar = fig.colorbar(pos, ticks=np.arange(0,n_clusters_ndvi), orientation='horizontal')
cbar.ax.set_xticklabels(['0','1','2'])  # horizontal colourbar
cbar.ax.set_xlabel('Cluster Number')
plt.ylabel('Latitude [degrees]')
plt.xlabel('Longitude [degrees]')
plt.show()

## K-Means Clustering (FDI)
Clustering across the floating debris index (FDI), which is defined in: Biermann, L., Clewley, D., Martinez-Vicente, V., and Topouzelis, K. (2020). Finding Plastic Patches in Coastal Waters using Optical Satellite Data. *Nature: Scientific Reports*, 10:5364. https://doi.org/10.1038/s41598-020-62298-z

In [ ]:
NIRprime = RE2+(SWIR1-RE2)*((833-665)/(1610.4-665))*10
fdi = NIR-NIRprime # floating debris index (FDI)
fdi_clust = fdi.reshape((-1, 1)) # reshape to allow clustering

# check for nodata
nd = np.isnan(fdi)
fdi[nd] = np.nanmean(fdi) # rough mean filter to allow for K-means clustering

# perform K-means clustering
kmeans_fdi = KMeans(n_clusters=n_clusters_fdi)
kmeans_fdi.fit(fdi_clust)
centroids_fdi = kmeans_fdi.cluster_centers_
labels_fdi = kmeans_fdi.labels_

# reshape output data for plotting
debris_fdi = np.choose(labels_fdi, centroids_fdi)
debris_fdi.shape = fdi.shape
labels_fdi.shape = fdi.shape

# plot results
fig, ax = plt.subplots(figsize=(8,8))
ax.set_title('FDI Classification')
pos = ax.imshow(labels_fdi, cmap=plt.get_cmap('viridis', n_clusters_fdi), extent=extent)
cbar = fig.colorbar(pos, ticks=np.arange(0,n_clusters_fdi), orientation='horizontal')
cbar.ax.set_xticklabels(['0','1','2'])  # horizontal colorbar
plt.ylabel('Latitude [degrees]')
plt.xlabel('Longitude [degrees]')
cbar.ax.set_xlabel('Cluster Number')
plt.show()

## Overlap NDVI-FDI Results
Produce a map that shows the areas that are defined in **both** the NDVI and FDI clustering approaches.

In [ ]:
# define the clusters to select for analysis
ndvi_clust = 1
fdi_clust = 2

In [ ]:
res_fdi = labels_fdi.reshape((-1, 1))
res_ndvi = labels_ndvi.reshape((-1, 1))

# reassign pixels
res = np.zeros(len(res_fdi),)
for i in range(len(res_fdi)):
    if res_fdi[i] == fdi_clust:
        if res_ndvi[i] == ndvi_clust:
            res[i] = 1
        else:
            res[i] = 0
    else:
        res[i] = 0

res.shape = fdi.shape

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
ax.set_title('Combined Classification of Floating Objects')
pos = ax.imshow(res, cmap=plt.get_cmap('Greys', 2), extent=extent)
cbar = fig.colorbar(pos, ticks=[0,1], orientation='horizontal')
cbar.ax.set_xticklabels(['Not Debris', 'Debris'])  # horizontal colorbar
plt.ylabel('Latitude [degrees]')
plt.xlabel('Longitude [degrees]')
cbar.ax.set_xlabel('Classification')
plt.show()

## Mask Land

In [ ]:
land = np.greater(SWIR1,BLU)
res[land] = "NaN"

fig, ax = plt.subplots(figsize=(8,8))
ax.set_title('Combined Classification of Floating Objects')
pos = ax.imshow(res, cmap=plt.get_cmap('Greys', 2), extent=extent)
cbar = fig.colorbar(pos, ticks=[0,1], orientation='horizontal')
cbar.ax.set_xticklabels(['Not Debris', 'Debris'])  # horizontal colorbar
plt.ylabel('Latitude [degrees]')
plt.xlabel('Longitude [degrees]')
cbar.ax.set_xlabel('Classification')
plt.show()

## Export Classification Raster

In [ ]:
# export raster
with rasterio.open(output_folder+'classified.tiff', 
                   'w',driver='GTiff',height=res.shape[0],width=res.shape[1],
                   count=1,dtype=res.dtype,crs=xda.rio.crs,nodata="NaN",transform=data.transform) as dst:dst.write(res, 1)

***
***
# Module 3: Create Shapefile from Classified Areas
Export classified areas as shapefiles.

Package installation requirements:
- Matplotlib (`matplotlib`): https://pypi.org/project/matplotlib/
- NumPy (`numpy`): https://pypi.org/project/numpy/
- Rasterio (`rasterio`): https://pypi.org/project/rasterio/
- Xarray (`xarray`): https://pypi.org/project/xarray/0.8.0rc1/
- Rasterio Xarray Extension (`rioxarray`): https://pypi.org/project/rioxarray/
- Shapely (`shapely`): https://pypi.org/project/shapely/
- Pandas (`pandas`): https://pypi.org/project/pandas/
- GeoPandas (`geopandas`): https://pypi.org/project/geopandas/
- Geocube (`geocube`): https://pypi.org/project/geocube/

***
## User Input
Define the key inputs for the module here:
1. Path to the dataset.
2. Data output path.

### Path to Data

In [ ]:
# define the path to the tiff output data from module 1 (or tiffs defined elsewhere)
try:
    req_id
except NameError:
    var_exists = False
    path_to_tiff = ''
else:
    var_exists = True

if var_exists:
    print('Found request ID generated from Module 1: '+req_id)
    path_to_tiff = f
else:
    if path_to_tiff == '':
        path_to_tiff = input('Please define the path to the GeoTIFF:')
    else:
        print('Path to GeoTIFF has been defined.')

## Output Path

In [ ]:
output_folder = ''
output_file = 'debris_outline.shp'

***
## Load Packages and Define Functions

In [ ]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import rioxarray
from shapely.geometry import shape
import geopandas as gpd
import pandas as pd
import shapely
from geocube.vector import vectorize

## Load Data

In [ ]:
# open tiff with rasterio
xda = rioxarray.open_rasterio(path_to_tiff)
print('GeoTIFF opened, '+str(len(xda.band))+' bands found.')

# define the spatial extent
lon = xda.x.data
lat = xda.y.data
extent = [np.min(lon), np.max(lon), np.max(lat), np.min(lat)]

xda

## Check Data

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
ax.set_title('Combined Classification of Floating Objects')
pos = ax.imshow(xda[0], cmap=plt.get_cmap('Greys', 2), extent=extent)
cbar = fig.colorbar(pos, ticks=[0,1], orientation='horizontal')
cbar.ax.set_xticklabels(['Not Debris', 'Debris'])  # horizontal colorbar
plt.ylabel('Latitude [degrees]')
plt.xlabel('Longitude [degrees]')
cbar.ax.set_xlabel('Classification')
plt.show()

## Create and Export Shapefile

In [ ]:
# read data and convert to the correct format
class_data = rioxarray.open_rasterio(path_to_tiff, mask_and_scale=True).squeeze()
class_data.name = "class"
class_data_i = class_data.astype(int)
class_gdf = vectorize(class_data_i)

In [ ]:
filtered_gdf = class_gdf[class_gdf['class'] == 1]
filtered_gdf

In [ ]:
# export shapefile
filtered_gdf.to_file(output_folder+output_file, driver='ESRI Shapefile') 